In [2]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
processed_data_path = '../data/processed/'
raw_data_path = '../data/raw/'

name = 'hd'

reviews_pro = pd.read_csv(processed_data_path + name + '_reviews.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')

display(resumme_raw)
display(reviews_pro.sample(5))

reviews = reviews_pro.copy()
resumme = resumme_raw.copy()

,stars,reviews
0,5,2290
1,4,1308
2,3,396
3,2,132
4,1,128


,review_id,review,local_guide_reviews,rating_score,service,meal_type,price_per_person_category,food_score,service_score,atmosphere_score,recommendations_list,date,avg_price_per_person
176,176,No es por la estupenda carta de cafetería (pes...,31.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,[''],2024-01-01,NaN
152,152,En nuestra #RutaDelGofre visitamos este mítico...,212.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,[''],2017-01-01,NaN
223,223,"¡Muy buen lugar! El servicio es increíble, par...",6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,[''],2022-01-01,NaN
228,228,¡Muy bien! Buena comida y buen servicio ☺️,227.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,[''],2023-01-01,NaN
166,166,El desayuno por el precio que tiene no nos ent...,504.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,[''],2023-01-01,NaN
